# Quantify organelle contacts - part 11.3
--------------------

## OVERVIEW
Now that all organelles and masks are segmented, we can begin to quantify features of organelle composition, morphology, contacts, and distribution. 


## OBJECTIVE: ✅ Quantify ***organelle*** contacts
In this notebook, the logic for quantifying organelle contact composition (how much of each contact is present) and morphology (contact site size and shape) is outlined.


## IMPORTS

In [2]:
# top level imports
from pathlib import Path
import os, sys
import itertools

import parse

import napari

from skimage.measure import (regionprops, _regionprops)

from napari.utils.notebook_display import nbscreenshot

### import local python functions in ../infer_subc
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from infer_subc.core.file_io import (read_czi_image,
                                        export_inferred_organelle,
                                        import_inferred_organelle,
                                        export_tiff,
                                        list_image_files)

from infer_subc.core.img import *
from infer_subc.utils.stats import *
from infer_subc.utils.stats import (_my_props_to_dict, _assert_uint16_labels)
from infer_subc.utils.stats_helpers import *

from infer_subc.organelles import * 

from infer_subc.constants import (TEST_IMG_N,
                                    NUC_CH ,
                                    LYSO_CH ,
                                    MITO_CH ,
                                    GOLGI_CH ,
                                    PEROX_CH ,
                                    ER_CH ,
                                    LD_CH ,
                                    RESIDUAL_CH ) 

import time
%load_ext autoreload
%autoreload 2

## Get and load image for quantification
Specifically, this will include the raw image and the outputs from segmentation.

In [3]:
test_img_n = TEST_IMG_N

data_root_path = Path(os.path.expanduser("~")) / "Documents/Python_Scripts/Infer-subc"

raw_data_path = data_root_path / "raw"
im_type = ".czi"

raw_file_list = list_image_files(raw_data_path,im_type)
raw_img_name = raw_file_list[test_img_n]

# adding an additional list of image paths for the matching segmentation files
seg_data_path = data_root_path / "out"
seg_file_list = list_image_files(seg_data_path, "tiff")

# changing output directory for this notebook to a new folder called "quant"
out_data_path = data_root_path / "quant"
if not Path.exists(out_data_path):
    Path.mkdir(out_data_path)
    print(f"making {out_data_path}")

In [4]:
# raw image
raw_img_data, raw_meta_dict = read_czi_image(raw_img_name)

channel_names = raw_meta_dict['name']
img = raw_meta_dict['metadata']['aicsimage']
scale = raw_meta_dict['scale']
channel_axis = raw_meta_dict['channel_axis']

In [5]:
## For each import, change the string to match the suffix on the segmentation files (i.e., the stuff following the "-")

# masks
nuc_seg = import_inferred_organelle("20230426_test_nuc", raw_meta_dict, seg_data_path)
cell_seg = import_inferred_organelle("20230426_test_cell", raw_meta_dict, seg_data_path)
cyto_seg = import_inferred_organelle("20230426_test_cyto", raw_meta_dict, seg_data_path)
# mask_seg = import_inferred_organelle("masks", raw_meta_dict, seg_data_path)

#organelles
lyso_seg = import_inferred_organelle("20230426_test_lyso", raw_meta_dict, seg_data_path)
mito_seg = import_inferred_organelle("20230426_test_mito", raw_meta_dict, seg_data_path)
golgi_seg = import_inferred_organelle("20230426_test_golgi", raw_meta_dict, seg_data_path)
perox_seg = import_inferred_organelle("20230426_test_perox", raw_meta_dict, seg_data_path)
ER_seg = import_inferred_organelle("20230426_test_ER", raw_meta_dict, seg_data_path)
LD_seg = import_inferred_organelle("20230426_test_LD", raw_meta_dict, seg_data_path)


loaded  inferred 3D `20230426_test_nuc`  from C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\out 
loaded  inferred 3D `20230426_test_cell`  from C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\out 
loaded  inferred 3D `20230426_test_cyto`  from C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\out 
loaded  inferred 3D `20230426_test_lyso`  from C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\out 
loaded  inferred 3D `20230426_test_mito`  from C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\out 
loaded  inferred 3D `20230426_test_golgi`  from C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\out 
loaded  inferred 3D `20230426_test_perox`  from C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\out 
loaded  inferred 3D `20230426_test_ER`  from C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\out 
loaded  inferred 3D `20230426_test_LD`  from C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\out 


-----------------
## **ORGANELLE CONTACTS**

Here we are defining an **organelle contact** as any amount of pixel/voxel overlap between two organelles of different types. The overlap region will be know as the **contact site**. We have implemented a pipeline that identifies two-way contacts, but in reality there exists higher order contacts as well (e.g., three-way, four-way, etc.). Quantification of high order contacts have not been implimented here.

We will utilize similar regionprops measurements as was done to quantify organelle and cell regions morphology.

> NOTE: Since biological contact sites are smaller than our resolution limit for confocal microscopy, the region of overlap is only an estimation of contacts.
>> It may be helpful to dilate a single organelle (the organelle of interest) before determining the overlap region. This will include any touching, but not overlapping interactions and may improve our ability to detect differences in this subresolution system.

### 0. Apply cell mask
To ensure we are performing single cell analysis, we will apply the cell segmentation as a mask.

In [6]:
lyso_masked = apply_mask(lyso_seg, cell_seg)
mito_masked = apply_mask(mito_seg, cell_seg)
golgi_masked = apply_mask(golgi_seg, cell_seg)
perox_masked = apply_mask(perox_seg, cell_seg)
ER_masked = apply_mask(ER_seg, cell_seg)
LD_masked = apply_mask(LD_seg, cell_seg)

### 1. Create region of overlap between organelle 'a' and organelle 'b'

Here we include two types of overlaps that have different biological relevance:
- **aXb**: organelle a overlapping with organelle b (*NOTE: aXb and bXa overlaps are redundant*)
- **shell_aXb**: the "shell", or membrane, of organelle a overlapping with organelle b (*NOTE: shell_aXb and shellbXa are *NOT* redundant*)

In [7]:
# creating aXb overlaps
a = _assert_uint16_labels(lyso_masked)
b = _assert_uint16_labels(mito_masked)

a_int_b = np.logical_and(a > 0, b > 0)

test_labels = label(apply_mask(a_int_b, cell_seg)).astype("int")

In [ ]:
## creating shell_aXb overlaps
## We need to conside how to do this correctly as creating a "shell" by eroding the object doesn't make sense 
## since our data is anisotropic

# a_shell_int_b = np.logical_and(np.logical_xor(a > 0, binary_erosion(a > 0)), b > 0)
# test_shell_labels = label(apply_mask(a_shell_int_b, cell_seg)).astype("int")

In [ ]:
## creating a function to measure the surface area of each object. This function utilizes "marching_cubes" to generate a mesh (non-pixelated object)
## the main issue I am running into here is that I can't figure out how to create the mesh with the same exact scale as the 

# from mpl_toolkits.mplot3d.art3d import Poly3DCollection

# def _surface_area_from_props_membrane_in_contact(labels: np.ndarray,
#                              props: dict,
#                              spacing: Union[tuple, None]=None):
#     """ 
#     a function for getting surface area of volumetric objects

#     Parameters:
#     ----------
#     lables:
#         the segmentation np.ndarray with each object labeled a different number
#     props:
#         region props dictionary resulting from the _my_props_to_dict() function
#     spacing:
#         tuple of the dimension lengths in the same order as the dimension of your np.ndarray labels input
#     """
    
#     if spacing is None:
#         spacing = np.ones(labels.ndim)

#     surface_areas = np.zeros(len(props["label"]))

#     for lab in np.unique(labels)[1:]:
#         volume = labels == lab
#         verts, faces, _normals, _values = marching_cubes(volume,
#                                                          method="lewiner",
#                                                          spacing=spacing,
#                                                          level=0)
        
#         mesh = Poly3DCollection(verts[faces])
        
#         # surface_areas[index] = mesh_surface_area(verts, faces)

#     return surface_areas

In [8]:
# nuc_masked = apply_mask(nuc_seg, cell_seg)

# test_list_of_objs = np.unique(nuc_masked)[1:]

# alt_new_test_volume = nuc_masked == test_list_of_objs[0]

# from mpl_toolkits.mplot3d.art3d import Poly3DCollection

# test_verts, test_faces, test__normals, test__values = marching_cubes(alt_new_test_volume,
#                                                     method="lewiner",
#                                                     spacing=scale,
#                                                     level=0)

# test_mesh = Poly3DCollection(test_verts[test_faces])


In [ ]:
# viewer.add_image(alt_new_test_volume, scale=scale)
# viewer.add_surface((test_verts, test_faces), scale=scale)

#### ALT OPTION: Include both the regular and shell contact measures for each contact site everytime.

In [ ]:
## The logic behind this is that for each contact site (region of overlap between unique organelles),
## we will also collect information from the "shell" or membrane from both organelles in that contact.

# a_int_b = np.logical_and(a > 0, b > 0)
# test_labels = label(apply_mask(a_int_b, cell_seg)).astype("int")

# a_shell_int_b = np.logical_and(np.logical_xor(a > 0, binary_erosion(a > 0)), b > 0)
# test_a_shell_labels = label(apply_mask(a_shell_int_b, cell_seg)).astype("int")

# b_shell_int_a = np.logical_and(np.logical_xor(b > 0, binary_erosion(b > 0)), a > 0)
# test_b_shell_labels = label(apply_mask(b_shell_int_a, cell_seg)).astype("int")

In [ ]:
# viewer = napari.Viewer()

In [ ]:
# viewer.add_image(test_labels, colormap='gray')
# viewer.add_image(test_a_shell_labels, colormap='green', opacity=0.5)
# viewer.add_image(test_b_shell_labels, colormap='red', opacity=0.5)

In [ ]:
# np.unique(test_labels), np.unique(test_a_shell_labels), np.unique(test_b_shell_labels)

### 2. Build the list of measurements we want to include from regionprops and run regionprops

In [27]:
# start with LABEL
test_Xproperties = ["label"]

# add position
test_Xproperties = test_Xproperties + ["centroid", "bbox"]

# add area
test_Xproperties = test_Xproperties + ["area", "equivalent_diameter"] # "num_pixels", 

# add shape measurements - NOTE: can't include minor axis measure because some of the contact sites are only one pixel
test_Xproperties = test_Xproperties + ["extent", "euler_number", "solidity", "axis_major_length", "slice"] # "feret_diameter_max",  , "axis_minor_length"

test_Xproperties

['label',
 'centroid',
 'bbox',
 'area',
 'equivalent_diameter',
 'extent',
 'euler_number',
 'solidity',
 'axis_major_length',
 'slice']

In [28]:
## NOTE: it may be worth out time to deal with the measurements that having issues with one voxel objects: 
## axis_min_length - the minimum axis of a voxel should be calulated as the minimum axis of a elipsoid with the same central moment as the voxel;
## when scaling this number should be calculated, but without scale, I think this would just be ~1

test_Xprops = regionprops_table(test_labels, 
                                intensity_image=None, 
                                properties=test_Xproperties, 
                                extra_properties=None, 
                                spacing=scale)

# # For testing adding the shell contacts to the main contacts table
# test_a_shell_Xprops = regionprops_table(test_a_shell_labels, intensity_image=None, properties=test_Xproperties, extra_properties=None)
# test_b_shell_Xprops = regionprops_table(test_b_shell_labels, intensity_image=None, properties=test_Xproperties, extra_properties=None)

In [29]:
test_Xsurface_area_tab = pd.DataFrame(surface_area_from_props(test_labels, test_Xprops))

### 3. Track which individual organelles are involved in that contact.

In [30]:
# collecting a list of organelle IDs associated to each contact site
test_label_a = []
test_index_ab = []
test_label_b = []
for index, lab in enumerate(test_Xprops["label"]):
    # this seems less elegant than you might wish, given that regionprops returns a slice,
    # but we need to expand the slice out by one voxel in each direction, or surface area freaks out
    volume = test_labels[test_Xprops["slice"][index]]
    la = a[test_Xprops["slice"][index]]
    lb = b[test_Xprops["slice"][index]]
    volume = volume == lab
    la = la[volume]
    lb = lb[volume]

    all_as = np.unique(la[la>0]).tolist()
    all_bs = np.unique(lb[lb>0]).tolist()
    if len(all_as) != 1:
        print(f"we have an error.  as-> {all_as}")
    if len(all_bs) != 1:
        print(f"we have an error.  bs-> {all_bs}")

    test_label_a.append(all_as[0] )
    test_label_b.append(all_bs[0] )
    test_index_ab.append(f"{all_as[0]}_{all_bs[0]}")

In [ ]:
# use_shell_a = False #True = use shell, False = no shell

# test_Xprops["label_A"] = test_label_a ## TODO: FIND A WAY TO INSERT ACTUAL ORGANELLE NAME, NOT "a" OR "b"
# test_Xprops["label_b"] = test_label_b
# test_Xprops_table = pd.DataFrame(test_Xprops)
# test_Xprops_table.insert(11, "surface_area", test_Xsurface_area_tab)
# test_Xprops_table.rename(columns={"area": "volume"}, inplace=True)
# test_Xprops_table.drop(columns="slice", inplace=True)
# test_Xprops_table.insert(loc=0,column='label_',value=test_index_ab) # do we need this is every row already has columns for each organelle and ID
# test_Xprops_table.insert(loc=0,column='shell',value=use_shell_a)

In [40]:
test_org_a = "lyso"
test_org_b = "mito"
# use_shell_a = False #True = use shell, False = no shell
# if use_shell_a is True:
#     test_Xprops_table.insert(loc=0,column='shell',value=use_shell_a)

test_Xprops_table = pd.DataFrame(test_Xprops)
test_Xprops_table.drop(columns=["slice", "label"], inplace=True)
test_Xprops_table.insert(0, "object", f"{test_org_a}X{test_org_b}")
test_Xprops_table.insert(0, 'label',value=test_index_ab)
test_Xprops_table.rename(columns={"area": "volume"}, inplace=True)

test_Xprops_table.insert(11, "surface_area", test_Xsurface_area_tab)
test_Xprops_table.insert(13, "SA_to_volume_raiot", test_Xprops_table["surface_area"].div(test_Xprops_table["volume"]))

test_scale = scale

if test_scale is not None:
    test_round_scale = (round(test_scale[0], 4), round(test_scale[1], 4), round(test_scale[2], 4))
    test_Xprops_table.insert(loc=2, column="scale", value=f"{test_round_scale}")
else: 
    test_Xprops_table.insert(loc=2, column="scale", value=f"{tuple(np.ones(test_label.ndim))}") 

test_Xprops_table

,label,object,scale,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5,surface_area,volume,SA_to_volume_raiot,equivalent_diameter,extent,euler_number,solidity,axis_major_length
0,3_2,lysoXmito,"(0.3891, 0.0799, 0.0799)",0.000000,22.670667,16.512820,0,279,203,1,289,210,56.715870,0.076954,737.013946,0.527728,0.442857,1,inf,0.792587
1,4_2,lysoXmito,"(0.3891, 0.0799, 0.0799)",0.345883,28.017201,22.932039,0,349,286,2,354,289,39.173515,0.022341,1753.407412,0.349438,0.300000,1,0.750000,0.642565
2,11_2,lysoXmito,"(0.3891, 0.0799, 0.0799)",0.661501,22.808681,15.875822,1,283,195,3,289,203,121.070824,0.074471,1625.739824,0.521991,0.312500,0,0.517241,1.063336
3,14_2,lysoXmito,"(0.3891, 0.0799, 0.0799)",0.933884,26.316822,20.384133,1,325,251,4,334,260,186.289124,0.223414,833.830529,0.752842,0.370370,1,0.769231,1.020799
4,26_2,lysoXmito,"(0.3891, 0.0799, 0.0799)",2.353893,23.447392,16.789077,2,281,201,9,305,225,739.604797,0.704994,1049.093464,1.104237,0.070437,1,0.250220,3.765527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,26_2,lysoXmito,"(0.3891, 0.0799, 0.0799)",4.669422,21.525410,18.130865,12,269,227,13,271,228,12.585057,0.004965,2534.883486,0.211657,1.000000,1,inf,0.178598
95,109_2,lysoXmito,"(0.3891, 0.0799, 0.0799)",5.280894,23.626795,23.195108,13,292,286,16,299,295,138.413239,0.104260,1327.581347,0.583946,0.222222,1,0.600000,1.247478
96,116_2,lysoXmito,"(0.3891, 0.0799, 0.0799)",5.836777,16.852919,16.293817,15,211,204,16,212,205,6.928203,0.002482,2790.958718,0.167992,1.000000,1,inf,0.000000
97,117_2,lysoXmito,"(0.3891, 0.0799, 0.0799)",5.836777,20.216403,16.675426,15,249,207,16,257,212,76.461182,0.044683,1711.202091,0.440265,0.450000,1,inf,0.845828


#### ALT OPTION: measure full and shell measurements for each contact site each time

In [ ]:
# test_Xprops_tab_combined = pd.DataFrame(test_Xprops)

# test_Xprops_tab_combined

In [ ]:
# test_org_a = "lyso"
# test_org_b = "mito"
# test_Xprops_tab_combined.insert(11, "surface_area", test_Xsurface_area_tab)
# test_Xprops_tab_combined.insert(1, "org_B_label", test_label_b)
# test_Xprops_tab_combined.insert(1, "org_B", test_org_b)
# test_Xprops_tab_combined.insert(1, "org_A_label", test_label_a)
# test_Xprops_tab_combined.insert(1, "org_A", test_org_a)
# test_Xprops_tab_combined.insert(0,column='X_label',value=test_index_ab)
# test_Xprops_tab_combined.rename(columns={"area": "volume"}, inplace=True)
# test_Xprops_tab_combined.drop(columns="slice", inplace=True)

# test_Xprops_tab_combined

In [ ]:
# test_a_shell_tab = pd.DataFrame(test_a_shell_Xprops)

# test_a_shell_tab.rename(columns={"area": "shell_a_volume"}, inplace=True)
# test_a_shell_tab.rename(columns={"equivalent_diameter": "shell_a_equivalent_diameter"}, inplace=True)
# test_a_shell_tab.rename(columns={"extent": "shell_a_extent"}, inplace=True)
# test_a_shell_tab.rename(columns={"euler_number": "shell_a_euler_number"}, inplace=True)
# test_a_shell_tab.rename(columns={"convex_area": "shell_a_convex_area"}, inplace=True)
# test_a_shell_tab.rename(columns={"solidity": "shell_a_solidity"}, inplace=True)
# test_a_shell_tab.rename(columns={"axis_major_length": "shell_a_axis_major_length"}, inplace=True)

# test_a_shell_tab

In [ ]:
# new_shell_a_tab = test_a_shell_tab.loc[:, 'shell_a_volume':'shell_a_axis_major_length']
# new_shell_a_tab.insert(0,column='X_label',value=test_index_ab)
# new_shell_a_tab

In [ ]:
# new_comboXtab = pd.merge(test_Xprops_tab_combined, new_shell_a_tab, on='X_label')
# new_comboXtab

In [ ]:
# test_b_shell_tab = pd.DataFrame(test_b_shell_Xprops)
# test_b_shell_tab

In [ ]:


# test_b_shell_tab.rename(columns={"area": "shell_b_volume"}, inplace=True)
# test_b_shell_tab.rename(columns={"equivalent_diameter": "shell_b_equivalent_diameter"}, inplace=True)
# test_b_shell_tab.rename(columns={"extent": "shell_b_extent"}, inplace=True)
# test_b_shell_tab.rename(columns={"euler_number": "shell_b_euler_number"}, inplace=True)
# test_b_shell_tab.rename(columns={"convex_area": "shell_b_convex_area"}, inplace=True)
# test_b_shell_tab.rename(columns={"solidity": "shell_b_solidity"}, inplace=True)
# test_b_shell_tab.rename(columns={"axis_major_length": "shell_b_axis_major_length"}, inplace=True)

# new_shell_b_tab = test_b_shell_tab.loc[:, 'shell_b_volume':'shell_b_axis_major_length']
# new_shell_b_tab.insert(0,column='X_label',value=test_index_ab)
# new_shell_b_tab

In [ ]:
# test_b_shell_tab = pd.DataFrame(test_b_shell_Xprops)

`TO DO: can we make the shell make more sense by using the mesh thing that we use for surface area?`

In [41]:
test_Xprops_table

,label,object,scale,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5,surface_area,volume,SA_to_volume_raiot,equivalent_diameter,extent,euler_number,solidity,axis_major_length
0,3_2,lysoXmito,"(0.3891, 0.0799, 0.0799)",0.000000,22.670667,16.512820,0,279,203,1,289,210,56.715870,0.076954,737.013946,0.527728,0.442857,1,inf,0.792587
1,4_2,lysoXmito,"(0.3891, 0.0799, 0.0799)",0.345883,28.017201,22.932039,0,349,286,2,354,289,39.173515,0.022341,1753.407412,0.349438,0.300000,1,0.750000,0.642565
2,11_2,lysoXmito,"(0.3891, 0.0799, 0.0799)",0.661501,22.808681,15.875822,1,283,195,3,289,203,121.070824,0.074471,1625.739824,0.521991,0.312500,0,0.517241,1.063336
3,14_2,lysoXmito,"(0.3891, 0.0799, 0.0799)",0.933884,26.316822,20.384133,1,325,251,4,334,260,186.289124,0.223414,833.830529,0.752842,0.370370,1,0.769231,1.020799
4,26_2,lysoXmito,"(0.3891, 0.0799, 0.0799)",2.353893,23.447392,16.789077,2,281,201,9,305,225,739.604797,0.704994,1049.093464,1.104237,0.070437,1,0.250220,3.765527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,26_2,lysoXmito,"(0.3891, 0.0799, 0.0799)",4.669422,21.525410,18.130865,12,269,227,13,271,228,12.585057,0.004965,2534.883486,0.211657,1.000000,1,inf,0.178598
95,109_2,lysoXmito,"(0.3891, 0.0799, 0.0799)",5.280894,23.626795,23.195108,13,292,286,16,299,295,138.413239,0.104260,1327.581347,0.583946,0.222222,1,0.600000,1.247478
96,116_2,lysoXmito,"(0.3891, 0.0799, 0.0799)",5.836777,16.852919,16.293817,15,211,204,16,212,205,6.928203,0.002482,2790.958718,0.167992,1.000000,1,inf,0.000000
97,117_2,lysoXmito,"(0.3891, 0.0799, 0.0799)",5.836777,20.216403,16.675426,15,249,207,16,257,212,76.461182,0.044683,1711.202091,0.440265,0.450000,1,inf,0.845828


### 4. Define `get_contact_metrics_3D()` function

Based on the _prototyping_ above define the function to quantify amount, size, and shape of organelles.

In [ ]:
def _get_contact_metrics_3D(a, a_name: str, b, b_name:str, mask, use_shell_a=False, include_distributions=False, nucleus_for_dist: Union[np.ndarray, None]=None):
    """
    collect volumentric measurements of organelle `a` intersect organelle `b`

    Parameters
    ------------
    a:
        a 3D np.ndarray image of one segemented organelle
    b:
        a 3D np.ndarray image of a second segemented organelle
    mask:
        a 3d np.ndarray image of the cell mask (or other mask of choice); used to create a "single cell" analysis
    use_shell_a:
        creates a "shell" of organelle a to simulate just the membrane area of the organelle and the performs the overlaps; all the same measurements are carried out of the shell region

    Returns
    -------------
    pandas dataframe of containing regionprops measurements (columns) for each overlap region between a and b (rows)

    Regionprops measurements included:
    ['label',
    'centroid',
    'bbox',
    'area',
    'equivalent_diameter',
    'extent',
    'feret_diameter_max',
    'euler_number',
    'convex_area',
    'solidity',
    'axis_major_length',
    'axis_minor_length']

    Additional measurement include:
    ['surface_area']
 
    """
    #########################
    ## CREATE OVERLAP REGIONS
    #########################
    a = _assert_uint16_labels(a)
    b = _assert_uint16_labels(b)

    if use_shell_a:
        a_int_b = np.logical_and(np.logical_xor(a > 0, binary_erosion(a > 0)), b > 0)
    else:
        a_int_b = np.logical_and(a > 0, b > 0)

    labels = label(apply_mask(a_int_b, mask)).astype("int")

    ##########################################
    ## CREATE LIST OF REGIONPROPS MEASUREMENTS
    ##########################################
    # start with LABEL
    properties = ["label"]

    # add position
    properties = properties + ["centroid", "bbox"]

    # add area
    properties = properties + ["area", "equivalent_diameter"] # "num_pixels", 

    # add shape measurements
    properties = properties + ["extent", "euler_number", "convex_area", "solidity", "axis_major_length", "slice"] # "feret_diameter_max", "axis_minor_length", 

    ##################
    ## RUN REGIONPROPS
    ##################
    props = regionprops_table(labels, intensity_image=None, properties=properties, extra_properties=None)

    ##################################################################
    ## RUN SURFACE AREA FUNCTION SEPARATELY AND APPEND THE PROPS_TABLE
    ##################################################################
    surface_area_tab = pd.DataFrame(surface_area_from_props(labels, props))

    ######################################################
    ## LIST WHICH ORGANELLES ARE INVOLVED IN THE CONTACTS
    ######################################################
    label_a = []
    index_ab = []
    label_b = []
    for index, lab in enumerate(props["label"]):
        # this seems less elegant than you might wish, given that regionprops returns a slice,
        # but we need to expand the slice out by one voxel in each direction, or surface area freaks out
        volume = labels[props["slice"][index]]
        la = a[props["slice"][index]]
        lb = b[props["slice"][index]]
        volume = volume == lab
        la = la[volume]
        lb = lb[volume]

        all_as = np.unique(la[la>0]).tolist()
        all_bs = np.unique(lb[lb>0]).tolist()
        if len(all_as) != 1:
            print(f"we have an error.  as-> {all_as}")
        if len(all_bs) != 1:
            print(f"we have an error.  bs-> {all_bs}")

        label_a.append(f"{all_as[0]}" )
        label_b.append(f"{all_bs[0]}" )
        index_ab.append(f"{all_as[0]}_{all_bs[0]}")

    props_table = pd.DataFrame(props)
    props_table.insert(11, "surface_area", surface_area_tab)
    props_table.insert(1, "org_B_label", label_b)
    props_table.insert(1, "org_B", b_name)
    props_table.insert(1, "org_A_label", label_a)
    props_table.insert(1, "org_A", a_name)
    if use_shell_a is True:
        props_table.insert(loc=0,column='shell',value=use_shell_a)
    props_table.insert(1,column='X_label',value=index_ab)
    props_table.rename(columns={"area": "volume"}, inplace=True)
    props_table.drop(columns="slice", inplace=True)

    # added option to measure contact distributions too
    if include_distributions is True:
        XY_contact_dist, zernike_contact_dist, XY_bin_masks = get_XY_distribution(cellmask_obj=mask, 
                                                                                  organelle_obj=a_int_b,
                                                                                  organelle_name=f"{a_name}X{b_name}",
                                                                                  nuclei_obj=nucleus_for_dist,
                                                                                  num_bins=5,
                                                                                  zernike_degrees=9)
        
        Z_contact_dist = get_Z_distribution(cellmask_obj=mask,
                                            organelle_obj=a_int_b,
                                            organelle_name=f"{a_name}X{b_name}",
                                            nuclei_obj=nucleus_for_dist)

        return props_table, XY_contact_dist, zernike_contact_dist, Z_contact_dist
    else:
        return props_table

In [ ]:
lysoXmito_stat_tab_test, test_cont_dist_XY, test_cont_dist_zern, test_cont_dist_Z = _get_contact_metrics_3D(a=lyso_seg, a_name=test_org_a, 
                                                                                                            b=mito_seg, b_name=test_org_b, 
                                                                                                            mask=cell_seg, 
                                                                                                            use_shell_a=False, include_distributions=True, nucleus_for_dist=nuc_seg)

lysoXmito_stat_tab_test

c:\Users\Shannon\anaconda3\envs\infer-subc\lib\site-packages\skimage\measure\_regionprops.py:431: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6013 qhull input error: input is less than 3-dimensional since all points have the same x coordinate    0

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 531759498  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _max-width  9  Error-roundoff 1.2e-14  _one-merge 8.5e-14
  _near-inside 4.2e-13  Visible-distance 2.4e-14  U-max-coplanar 2.4e-14
  Width-outside 4.8e-14  _wide-facet 1.5e-13  _maxoutside 9.7e-14

  return convex_hull_image(self.image)
c:\Users\Shannon\anaconda3\envs\infer-subc\lib\site-packages\skimage\measure\_regionprops.py:431: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6013 qhull input error: input is less than 3-dimensional since all points have the same x coordinate    0

While exe

,label,X_label,org_A,org_A_label,org_B,org_B_label,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5,volume,surface_area,equivalent_diameter,extent,euler_number,convex_area,solidity,axis_major_length
0,1,3_2,lyso,3,mito,2,0.000000,283.838710,206.741935,0,279,203,1,289,210,31.0,56.715870,3.897514,0.442857,1,0.0,inf,9.923254
1,2,4_2,lyso,4,mito,2,0.888889,350.777778,287.111111,0,349,286,2,354,289,9.0,39.173515,2.580762,0.300000,1,12.0,0.750000,5.314464
2,3,11_2,lyso,11,mito,2,1.700000,285.566667,198.766667,1,283,195,3,289,203,30.0,121.070824,3.855146,0.312500,0,58.0,0.517241,10.382515
3,4,14_2,lyso,14,mito,2,2.400000,329.488889,255.211111,1,325,251,4,334,260,90.0,186.289124,5.560083,0.370370,1,117.0,0.769231,10.087265
4,5,26_2,lyso,26,mito,2,6.049296,293.563380,210.200704,2,281,201,9,305,225,284.0,739.604797,8.155299,0.070437,1,1135.0,0.250220,34.962384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,95,26_2,lyso,26,mito,2,12.000000,269.500000,227.000000,12,269,227,13,271,228,2.0,12.585057,1.563185,1.000000,1,0.0,inf,2.236068
95,96,109_2,lyso,109,mito,2,13.571429,295.809524,290.404762,13,292,286,16,299,295,42.0,138.413239,4.312710,0.222222,1,70.0,0.600000,10.146311
96,97,116_2,lyso,116,mito,2,15.000000,211.000000,204.000000,15,211,204,16,212,205,1.0,6.928203,1.240701,1.000000,1,0.0,inf,0.000000
97,98,117_2,lyso,117,mito,2,15.000000,253.111111,208.777778,15,249,207,16,257,212,18.0,76.461182,3.251556,0.450000,1,0.0,inf,10.589834


In [ ]:
lysoXmito_stat_tab_test.equals(test_Xprops_table)

False

In [ ]:
test_Xprops_table

,label,X_label,org_A,org_A_label,org_B,org_B_label,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5,volume,surface_area,equivalent_diameter,extent,euler_number,convex_area,solidity,axis_major_length
0,1,3_2,lyso,3,mito,2,0.000000,283.838710,206.741935,0,279,203,1,289,210,31.0,56.715870,3.897514,0.442857,1,0.0,inf,9.923254
1,2,4_2,lyso,4,mito,2,0.888889,350.777778,287.111111,0,349,286,2,354,289,9.0,39.173515,2.580762,0.300000,1,12.0,0.750000,5.314464
2,3,11_2,lyso,11,mito,2,1.700000,285.566667,198.766667,1,283,195,3,289,203,30.0,121.070824,3.855146,0.312500,0,58.0,0.517241,10.382515
3,4,14_2,lyso,14,mito,2,2.400000,329.488889,255.211111,1,325,251,4,334,260,90.0,186.289124,5.560083,0.370370,1,117.0,0.769231,10.087265
4,5,26_2,lyso,26,mito,2,6.049296,293.563380,210.200704,2,281,201,9,305,225,284.0,739.604797,8.155299,0.070437,1,1135.0,0.250220,34.962384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,95,26_2,lyso,26,mito,2,12.000000,269.500000,227.000000,12,269,227,13,271,228,2.0,12.585057,1.563185,1.000000,1,0.0,inf,2.236068
95,96,109_2,lyso,109,mito,2,13.571429,295.809524,290.404762,13,292,286,16,299,295,42.0,138.413239,4.312710,0.222222,1,70.0,0.600000,10.146311
96,97,116_2,lyso,116,mito,2,15.000000,211.000000,204.000000,15,211,204,16,212,205,1.0,6.928203,1.240701,1.000000,1,0.0,inf,0.000000
97,98,117_2,lyso,117,mito,2,15.000000,253.111111,208.777778,15,249,207,16,257,212,18.0,76.461182,3.251556,0.450000,1,0.0,inf,10.589834


### Add prototype function into `stats.py`

In [ ]:
from infer_subc.utils.stats import get_contact_metrics_3D

lysoXmito_stat_final, t_XY_d, t_xern_d, t_Z_d = get_contact_metrics_3D(a=lyso_seg,
                                              a_name=test_org_a,
                                              b=mito_seg,
                                              b_name=test_org_b,
                                              mask=cell_seg,
                                              use_shell_a=False,
                                               include_distributions=True, nucleus_for_dist=nuc_seg)

c:\Users\Shannon\anaconda3\envs\infer-subc\lib\site-packages\skimage\measure\_regionprops.py:431: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6013 qhull input error: input is less than 3-dimensional since all points have the same x coordinate    0

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 536919247  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _max-width  9  Error-roundoff 1.2e-14  _one-merge 8.5e-14
  _near-inside 4.2e-13  Visible-distance 2.4e-14  U-max-coplanar 2.4e-14
  Width-outside 4.8e-14  _wide-facet 1.5e-13  _maxoutside 9.7e-14

  return convex_hull_image(self.image)
c:\Users\Shannon\anaconda3\envs\infer-subc\lib\site-packages\skimage\measure\_regionprops.py:431: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6013 qhull input error: input is less than 3-dimensional since all points have the same x coordinate    0

While exe

In [ ]:
lysoXmito_stat_final.equals(lysoXmito_stat_tab_test)

True

In [ ]:
lysoXmito_stat_final

,label,X_label,org_A,org_A_label,org_B,org_B_label,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5,volume,surface_area,equivalent_diameter,extent,euler_number,convex_area,solidity,axis_major_length
0,1,3_2,lyso,3,mito,2,0.000000,283.838710,206.741935,0,279,203,1,289,210,31.0,56.715870,3.897514,0.442857,1,0.0,inf,9.923254
1,2,4_2,lyso,4,mito,2,0.888889,350.777778,287.111111,0,349,286,2,354,289,9.0,39.173515,2.580762,0.300000,1,12.0,0.750000,5.314464
2,3,11_2,lyso,11,mito,2,1.700000,285.566667,198.766667,1,283,195,3,289,203,30.0,121.070824,3.855146,0.312500,0,58.0,0.517241,10.382515
3,4,14_2,lyso,14,mito,2,2.400000,329.488889,255.211111,1,325,251,4,334,260,90.0,186.289124,5.560083,0.370370,1,117.0,0.769231,10.087265
4,5,26_2,lyso,26,mito,2,6.049296,293.563380,210.200704,2,281,201,9,305,225,284.0,739.604797,8.155299,0.070437,1,1135.0,0.250220,34.962384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,95,26_2,lyso,26,mito,2,12.000000,269.500000,227.000000,12,269,227,13,271,228,2.0,12.585057,1.563185,1.000000,1,0.0,inf,2.236068
95,96,109_2,lyso,109,mito,2,13.571429,295.809524,290.404762,13,292,286,16,299,295,42.0,138.413239,4.312710,0.222222,1,70.0,0.600000,10.146311
96,97,116_2,lyso,116,mito,2,15.000000,211.000000,204.000000,15,211,204,16,212,205,1.0,6.928203,1.240701,1.000000,1,0.0,inf,0.000000
97,98,117_2,lyso,117,mito,2,15.000000,253.111111,208.777778,15,249,207,16,257,212,18.0,76.461182,3.251556,0.450000,1,0.0,inf,10.589834
